# Browser-Use SDK를 활용한 라이브 뷰 브라우저 도구

## 개요

이 튜토리얼에서는 Browser-Use를 사용하여 Amazon Bedrock Agentcore Browser 도구와 상호작용하고 브라우저를 실시간으로 확인하는 방법을 알아봅니다.

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형 | 대화형 |
| 에이전트 유형 | 단일 |
| Agentic Framework | Browser-Use |
| LLM 모델 | Anthropic Claude 3.7 Sonnet |
| 튜토리얼 구성 요소 | Browser-Use를 사용하여 브라우저 도구와 상호작용하고 실시간으로 확인 |
| 튜토리얼 수직 | 수직 |
| 예제 복잡성 | 쉬움 |
| 사용된 SDK | Amazon BedrockAgentCore Python SDK, Browser-Use |

### 튜토리얼 아키텍처

이 튜토리얼에서는 Browser-Use를 브라우저 도구와 함께 사용하고 브라우저를 실시간으로 확인하는 방법을 설명합니다.

이 예시에서는 Browse-Use 에이전트에 자연어 명령을 전송하여 Bedrock Agentcore 브라우저에서 작업을 수행하고 브라우저를 실시간으로 확인하도록 하겠습니다.

<div style="text-align:left">
<img src="images/browser-tool.png" width="50%"/>
</div>

### 튜토리얼 주요 기능

* 브라우저 도구 사용 및 실시간 확인
* 브라우저 도구와 함께 Browser-Use 사용

## Prerequisites

To execute this tutorial you will need:
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Browser-Use SDK 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Bedrock Agentcore 브라우저 도구와 라이브 뷰를 함께 사용하는 Browser-Use

여기서는 Amazon DCV SDK를 통해 Bedrock Agentcore 브라우저 도구에 연결하기 위해 헬퍼 클래스 `BrowserViewerServer`를 사용합니다.

In [ ]:
%%writefile live_view_with_browser_use.py
from browser_use import Agent
from browser_use.browser.session import BrowserSession
from bedrock_agentcore.tools.browser_client import BrowserClient
from browser_use.browser import BrowserProfile
from langchain_aws import ChatBedrockConverse
from rich.console import Console
from rich.panel import Panel
from contextlib import suppress
import argparse
import sys
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer
import asyncio
from boto3.session import Session

console = Console()

boto_session = Session()
region = boto_session.region_name


async def run_browser_task(
    browser_session: BrowserSession, bedrock_chat: ChatBedrockConverse, task: str
) -> None:
    """
    Run a browser automation task using browser_use

    Args:
        browser_session: Existing browser session to reuse
        bedrock_chat: Bedrock chat model instance
        task: Natural language task for the agent
    """
    try:
        # Show task execution
        console.print(f"\n[bold blue]🤖 Executing task:[/bold blue] {task}")

        # Create and run the agent
        agent = Agent(task=task, llm=bedrock_chat, browser_session=browser_session)

        # Run with progress indicator
        with console.status(
            "[bold green]Running browser automation...[/bold green]", spinner="dots"
        ):
            await agent.run()

        console.print("[bold green]✅ Task completed successfully![/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error during task execution:[/bold red] {str(e)}")
        import traceback

        if console.is_terminal:
            traceback.print_exc()


async def live_view_with_browser_use(prompt, region="us-west-2"):
    """
    Main function that demonstrates live browser viewing with Agent automation.

    Workflow:
    1. Creates Amazon Bedrock AgentCore browser client in us-west-2 region
    2. Waits for browser initialization (10-second required delay)
    3. Starts DCV-based live viewer server on port 8000 with browser control
    4. Configures multiple display size options (720p to 1440p)
    5. Establishes browser session for AI agent automation via CDP WebSocket
    6. Executes AI-driven tasks using Claude 3.7 Sonnet model
    7. Properly closes all sessions and stops browser client

    Features:
    - Real-time browser viewing through web interface
    - Manual take/release control functionality
    - AI automation with browser-use library
    - Configurable display layouts and sizes
    """
    console.print(
        Panel(
            "[bold cyan]Browser Live Viewer[/bold cyan]\n\n"
            "This demonstrates:\n"
            "• Live browser viewing with DCV\n"
            "• Configurable display sizes (not limited to 900×800)\n"
            "• Proper display layout callbacks\n\n"
            "[yellow]Note: Requires Amazon DCV SDK files[/yellow]",
            title="Browser Live Viewer",
            border_style="blue",
        )
    )

    try:
        # Step 1: Create browser session
        client = BrowserClient(region)
        client.start()
        
        ws_url, headers = client.generate_ws_headers()

        # Step 2: Start viewer server
        console.print("\n[cyan]Step 3: Starting viewer server...[/cyan]")
        viewer = BrowserViewerServer(client, port=8000)
        viewer_url = viewer.start(open_browser=True)

        # Step 3: Show features
        console.print("\n[bold green]Viewer Features:[/bold green]")
        console.print(
            "• Default display: 1600×900 (configured via displayLayout callback)"
        )
        console.print("• Size options: 720p, 900p, 1080p, 1440p")
        console.print("• Real-time display updates")
        console.print("• Take/Release control functionality")

        console.print("\n[yellow]Press Ctrl+C to stop[/yellow]")

        # Step 4: Use browser-use to interact with browser
        # Create persistent browser session and model
        browser_session = None
        bedrock_chat = None

        try:
            # Create browser profile with headers
            browser_profile = BrowserProfile(
                headers=headers,
                timeout=1500000,  # 150 seconds timeout
            )

            # Create a browser session with CDP URL and keep_alive=True for persistence
            browser_session = BrowserSession(
                cdp_url=ws_url,
                browser_profile=browser_profile,
                keep_alive=True,  # Keep browser alive between tasks
            )

            # Initialize the browser session
            console.print("[cyan]🔄 Initializing browser session...[/cyan]")
            await browser_session.start()

            # Create ChatBedrockConverse once
            bedrock_chat = ChatBedrockConverse(
                model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
                region_name=region,
            )

            console.print(
                "[green]✅ Browser session initialized and ready for tasks[/green]\n"
            )

            task = prompt

            await run_browser_task(browser_session, bedrock_chat, task)

        finally:
            # Close the browser session
            if browser_session:
                console.print("\n[yellow]🔌 Closing browser session...[/yellow]")
                with suppress(Exception):
                    await browser_session.close()
                console.print("[green]✅ Browser session closed[/green]")
   
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="Browser Search instruction")
    parser.add_argument("--region", default="us-west-2", help="AWS region")
    args = parser.parse_args()

    asyncio.run(live_view_with_browser_use(
        args.prompt, args.region
    ))

#### 스크립트 실행
아래 스크립트를 실행하세요. 스크립트 실행 후(프롬프트의 복잡도에 따라 다소 시간이 걸릴 수 있습니다), 출력을 스크롤하여 수행된 작업의 결과를 확인하세요.

In [ ]:
!python live_view_with_browser_use.py --prompt "Search for macbooks on amazon.com and extract the details of the first one" 

## 백그라운드에서 무슨 일이 일어났을까요?
* 브라우저 클라이언트를 인스턴스화하고 세션을 시작했습니다.
* 그런 다음 `BrowserViewerServer`를 사용하여 브라우저 세션에 연결하여 로컬에서 세션을 확인했습니다.
* 그런 다음 Browser-Use 에이전트를 생성하고 브라우저 세션의 세부 정보를 전달했습니다.
* 그런 다음 Browser-Use 에이전트에 자연어 명령을 전송하고 동작을 실시간으로 확인했습니다.

# Congratulations! 